<a href="https://colab.research.google.com/github/951237/TIL/blob/master/TIL__200601_%EC%BD%94%EB%A1%9C%EB%82%98_%ED%98%84%ED%99%A9_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_feat_%EC%9D%8C%EC%84%B1%EC%A7%80%EC%9B%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pyttsx3

In [0]:
import requests, json
import pyttsx3
import re, threading, time

In [0]:
API_KEY = " "

In [0]:
PROJECT_TOKEN = ""

In [0]:
RUN_TOKEN = ""

In [0]:
class Data():
    def __init__(self, api_key, project_token):
        self.api_key = api_key
        self.project_token = project_token
        self.param = {
            "api_key" : self.api_key
        }
        self.data = self.get_data()
    
    def get_data(self):
        response = requests.get(f'https://')